In [2]:
import json
import httpx
import string
from fuzzywuzzy import fuzz
from num2words import num2words
from googletrans import Translator
from tqdm.autonotebook import tqdm

/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [3]:
timeout = httpx.Timeout(30) 
gg_translator = Translator(timeout=timeout)
vi_num2words_dict = {
    0: 'không',
    1: 'một',
    2: 'hai',
    3: 'ba',
    4: 'bốn',
    5: 'năm',
    6: 'sáu',
    7: 'bảy',
    8: 'tám',
    9: 'chín',
    10: 'mười'
}
vi_words2num_dict = {
    'không': 0,
    'một': 1,
    'hai': 2,
    'ba': 3,
    'bốn': 4,
    'năm': 5,
    'sáu': 6,
    'bảy': 7,
    'tám': 8,
    'chín': 9,
    'mười': 10
}

def remove_punc(text):
    text = text.lower()
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)


def get_samples_by_id(image_id, annotations):
    return_samples = []
    for anno in annotations:
        if anno['image_id'] == image_id:
            return_samples.append(anno)

    return return_samples

In [11]:
group_questions = []

with open('./data/test/evjvqa_public_test-lang-qtype-answer.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
    
test_annotations = test_data['annotations']


with open('./outputs/results-ofa-huge-en.json', 'r', encoding='utf-8') as f:
    ofa = json.load(f)


with open('./outputs/results-mt5-vit.json', 'r', encoding='utf-8') as f:
    mt5vit = json.load(f)

In [10]:
for i in tqdm(range(len(test_annotations))):
    anno = test_annotations[i]
    
    if anno['question_type'] == 'HOW_MANY':
        if anno['language'] == 'en':
            answer = ofa[str(i)]
        else:
            current_answer = mt5vit[str(i)]
            tokens = current_answer.split()
            
            gr_samples = get_samples_by_id(anno['image_id'], annotations)
            en_question = gg_translator.translate(anno['question'], src=anno['language'], dest='en').text
            en_question = en_question.lower()

            max_score = 0
            similar_sample = None
            for sample in gr_samples:                    
                if sample['language'] == 'en':
                    fuzze_score = fuzz.partial_ratio(en_question, sample['question'])
                    if fuzze_score > max_score:
                        similar_sample = sample
                        max_score = fuzze_score

            if max_score >= 85 and similar_sample:
                tmp_answer = ofa[str(similar_sample['id'])]
                answer = gg_translator.translate(tmp_answer, src='en', dest=anno['language']).text
                if anno['language'] == 'vi':
                    tmp = vi_num2words_dict[int(answer)]
                    for j in range(len(tokens)):
                        if tokens[j] in vi_words2num_dict:
                            tokens[j] = tmp

                    answer = ' '.join(tokens)

            else:
                raise ValueError(f"This question '{anno['question']}' does not have a similar sentence.")

  0%|          | 0/5015 [00:00<?, ?it/s]

KeyError: '1705'